# Basic example of use of simulated annealing

Let us consider the one-dimensional Ising model of an antiferromagnet described by the Hamiltonian 

$$ H = J \sum_{\langle i, j \rangle} \sigma_i \sigma_j$$

where $\langle i, j \rangle$ denotes nearest neighbour sites, magnetic coupling J > 0 for antiferromagnetism and $\sigma_i$ is the respective Pauli spin matrix acting on site $i$.

We want to use the simulated annealing implemented on the QLM to solve this problem. The following example presents how one can do so.

## Encoding and solving the problem

In [ ]:
import numpy as np
from qat.qpus import SimulatedAnnealing
from qat.core import Observable, Schedule, Variable, Term

# Define the the problem
n_spins = 100
mag_coupling = 1  # >0 is antiferromagnetic, <0 is ferromagnetic
    
# Create a temperature function
t = Variable("t", float)
temp_t = 2 * (1 - t) + 0.001  # annealing requires going from a high to a very low temperature

# Create a QPU
n_steps = 500
qpu = SimulatedAnnealing(temp_t=temp_t,
                         n_steps=n_steps,
                         seed=817)

# Create an observable
observable = Observable(n_spins, pauli_terms=[Term(mag_coupling, "ZZ", [i, i + 1])
                                              for i in range(n_spins - 1)])

# Create a schedule for the annealing, a job, and send it to the QPU
drive = [(1, observable)]
schedule = Schedule(drive=drive)
job = schedule.to_job(nbshots=1)
result = qpu.submit(job)

# Show the solution
for sample in result:
    print(sample.state)

## Solution

We can see that as expected for an antiferromagnet, the resulting spin configuration is alternating spins up and spins down which translates to alternating $|0>$ and $|1>$ qubit states.